## Recordando SQL 

En esta sesión buscamos recordar SQL empleando una base de datos en la nube (AWS RDS). 

Para empezar importe el paquete psycopg2 y conéctese a la base de datos remota a través del puerto 5432. 


In [ ]:
!conda install psycopg2

In [1]:
import psycopg2

In [16]:
engine = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="juanf.c2ejf18x6i7a.us-east-1.rds.amazonaws.com",
    port='5432'
)

Para ejecutar consultas debe crear un cursor sobre la conexión.

In [17]:
cursor = engine.cursor()

La base de datos es comúnmente conocida como world y contiene 3 tablas: city, country y country_language. 

Los atributos de cada una de estas tablas es el siguiente:
* city:
  * (id, name, country_code, district, population)
* country:
  * (code, name, continent, region, surface_area. indep_year, population. life_expectancy, gnp, gnp_old, local_name, government_form, head_of_state, capital, code2)
* country_language
  * (country_code, language, is_official, percentage)

Con esta información realicemos algunas consultas. Revisemos por ejemplos las primeras 5 filas de la tabla city


In [4]:
query = "select * from city limit 5"
cursor.execute(query)
result = cursor.fetchall()
result

[(1, 'Kabul', 'AFG', 'Kabol', 1780000),
 (2, 'Qandahar', 'AFG', 'Qandahar', 237500),
 (3, 'Herat', 'AFG', 'Herat', 186800),
 (4, 'Mazar-e-Sharif', 'AFG', 'Balkh', 127800),
 (5, 'Amsterdam', 'NLD', 'Noord-Holland', 731200)]

Note que el resultado se almacena en result al ejecutar fetchall(). Una forma alternativa de imprimir cada fila/registro resultado de la consulta es el siguiente

In [5]:
for row in result:
    print(row)

(1, 'Kabul', 'AFG', 'Kabol', 1780000)
(2, 'Qandahar', 'AFG', 'Qandahar', 237500)
(3, 'Herat', 'AFG', 'Herat', 186800)
(4, 'Mazar-e-Sharif', 'AFG', 'Balkh', 127800)
(5, 'Amsterdam', 'NLD', 'Noord-Holland', 731200)


Si en algún momento se genera un error, la conexión queda bloqueada con una transacción que no terminó. Para descartar la transacción actual y volver a intentar ejecute un commit sobre la conexión.

In [22]:
engine.commit()


Contemos ahora el número de registros en cada tabla

In [7]:
query = "select count(*) from city"
cursor.execute(query)
result = cursor.fetchall()
result

[(4079,)]

In [8]:
query = "select count(*) from country"
cursor.execute(query)
result = cursor.fetchall()
result

[(239,)]

In [19]:
query = "select count(*) from country_language"
cursor.execute(query)
result = cursor.fetchall()
result

[(984,)]

Usando la tabla country, determinemos los continentes que aparecen en la tabla. 

In [9]:
query = "select distinct(continent) from country"
cursor.execute(query)
result = cursor.fetchall()
result

[('Asia',),
 ('South America',),
 ('North America',),
 ('Oceania',),
 ('Antarctica',),
 ('Africa',),
 ('Europe',)]

Fijémonos en el continente Europe y listemos los nombres y códigos de los países en este continente. 

In [22]:
query = "select code, name from country where continent='Europe'"
cursor.execute(query)
result = cursor.fetchall()
result

[('NLD', 'Netherlands'),
 ('ALB', 'Albania'),
 ('AND', 'Andorra'),
 ('BEL', 'Belgium'),
 ('BIH', 'Bosnia and Herzegovina'),
 ('GBR', 'United Kingdom'),
 ('BGR', 'Bulgaria'),
 ('ESP', 'Spain'),
 ('FRO', 'Faroe Islands'),
 ('GIB', 'Gibraltar'),
 ('SJM', 'Svalbard and Jan Mayen'),
 ('IRL', 'Ireland'),
 ('ISL', 'Iceland'),
 ('ITA', 'Italy'),
 ('AUT', 'Austria'),
 ('YUG', 'Yugoslavia'),
 ('GRC', 'Greece'),
 ('HRV', 'Croatia'),
 ('LVA', 'Latvia'),
 ('LIE', 'Liechtenstein'),
 ('LTU', 'Lithuania'),
 ('LUX', 'Luxembourg'),
 ('MKD', 'Macedonia'),
 ('MLT', 'Malta'),
 ('MDA', 'Moldova'),
 ('MCO', 'Monaco'),
 ('NOR', 'Norway'),
 ('PRT', 'Portugal'),
 ('POL', 'Poland'),
 ('FRA', 'France'),
 ('ROM', 'Romania'),
 ('SWE', 'Sweden'),
 ('DEU', 'Germany'),
 ('SMR', 'San Marino'),
 ('SVK', 'Slovakia'),
 ('SVN', 'Slovenia'),
 ('FIN', 'Finland'),
 ('CHE', 'Switzerland'),
 ('DNK', 'Denmark'),
 ('CZE', 'Czech Republic'),
 ('UKR', 'Ukraine'),
 ('HUN', 'Hungary'),
 ('BLR', 'Belarus'),
 ('VAT', 'Holy See (Vatican

Determine los 5 países con mayor población de Europa. 

In [23]:
query = """
select name, population  from country 
where continent='Europe'  
order by population
desc
limit 5"""
cursor.execute(query)
result = cursor.fetchall()
result

[('Russian Federation', 146934000),
 ('Germany', 82164700),
 ('United Kingdom', 59623400),
 ('France', 59225700),
 ('Italy', 57680000)]

In [30]:
engine.commit()

Determine la población promedio de Europa.

In [11]:
query = """
select avg(population) from country 
where continent='Europe'  
"""
cursor.execute(query)
result = cursor.fetchall()
result

[(Decimal('15871186.956521739130'),)]

Liste los países que tienen un población superior al promedio del continente. Para cada país incluya nombre y población.

In [31]:
query = """
select name, population from
country 
where continent='Europe' and 
population >
(
 select avg(population) from country 
 where continent='Europe' 
) 
order by population
desc
"""
cursor.execute(query)
result = cursor.fetchall()
result

[('Russian Federation', 146934000),
 ('Germany', 82164700),
 ('United Kingdom', 59623400),
 ('France', 59225700),
 ('Italy', 57680000),
 ('Ukraine', 50456000),
 ('Spain', 39441700),
 ('Poland', 38653600),
 ('Romania', 22455500)]

Determine los tipos de gobierno existentes en Europa.

In [32]:
query = """
select distinct(government_form) from country
where continent = 'Europe'
"""
cursor.execute(query)
result = cursor.fetchall()
result


[('Republic',),
 ('Dependent Territory of Norway',),
 ('Parliamentary Coprincipality',),
 ('Part of Denmark',),
 ('Constitutional Monarchy',),
 ('Federal Republic',),
 ('Constitutional Monarchy, Federation',),
 ('Dependent Territory of the UK',),
 ('Independent Church State',),
 ('Federation',)]

Determine los países europeos que tienen un tipo de gobierno entre los tres más comunes en el continente. Para cada país incluya nombre y población. 

In [34]:
query = """
WITH A AS 
(select government_form, count(government_form) as num_paises from country
 where continent = 'Europe'
 group by government_form
 order by num_paises
 desc
 limit 3
)
select name, population
from country inner join A on 
country.government_form = A.government_form
where continent = 'Europe'
order by population
desc
"""
cursor.execute(query)
result = cursor.fetchall()
result


[('Russian Federation', 146934000),
 ('Germany', 82164700),
 ('United Kingdom', 59623400),
 ('France', 59225700),
 ('Italy', 57680000),
 ('Ukraine', 50456000),
 ('Spain', 39441700),
 ('Poland', 38653600),
 ('Romania', 22455500),
 ('Netherlands', 15864000),
 ('Yugoslavia', 10640000),
 ('Greece', 10545700),
 ('Czech Republic', 10278100),
 ('Belarus', 10236000),
 ('Hungary', 10043200),
 ('Portugal', 9997600),
 ('Sweden', 8861400),
 ('Bulgaria', 8190900),
 ('Austria', 8091800),
 ('Slovakia', 5398700),
 ('Denmark', 5330000),
 ('Finland', 5171300),
 ('Norway', 4478500),
 ('Croatia', 4473000),
 ('Moldova', 4380000),
 ('Bosnia and Herzegovina', 3972000),
 ('Ireland', 3775100),
 ('Lithuania', 3698500),
 ('Albania', 3401200),
 ('Latvia', 2424200),
 ('Macedonia', 2024000),
 ('Slovenia', 1987800),
 ('Estonia', 1439200),
 ('Luxembourg', 435700),
 ('Malta', 380200),
 ('Iceland', 279000),
 ('Monaco', 34000),
 ('Liechtenstein', 32300),
 ('San Marino', 27000)]

Liste cada país con su capital, población total, población de la capital y fracción de población que vive en la capital. Reporte el top 20 de los países con mayor y menor fracción de población en la capital. Por motivos de legibilidad, se espera que la fracción se muestre como un número con 4 posiciones decimales. 

Considere el uso de CAST AS y ROUND. 

In [34]:
query = """

WITH A AS
(select name, population, capital
from country)

(select A.name, city.name, A.population, city.population, round(city.population/ CAST(A.population as numeric),4) as fraccion
from city inner join A on
city.id = A.capital
order by fraccion
desc
limit 20)
union

((select A.name, city.name, A.population, city.population, round(city.population/ CAST(A.population as numeric),4) as fraccion
from city inner join A on
city.id = A.capital
order by fraccion
asc
limit 20))

order by fraccion
desc


"""
cursor.execute(query)
result = cursor.fetchall()
result



[('Singapore', 'Singapore', 3567000, 4017733, Decimal('1.1264')),
 ('Gibraltar', 'Gibraltar', 25000, 27025, Decimal('1.0810')),
 ('Macao', 'Macao', 473000, 437500, Decimal('0.9249')),
 ('Pitcairn', 'Adamstown', 50, 42, Decimal('0.8400')),
 ('Saint Pierre and Miquelon', 'Saint-Pierre', 7000, 5808, Decimal('0.8297')),
 ('Falkland Islands', 'Stanley', 2000, 1636, Decimal('0.8180')),
 ('Palau', 'Koror', 19000, 12000, Decimal('0.6316')),
 ('Djibouti', 'Djibouti', 638000, 383000, Decimal('0.6003')),
 ('Cook Islands', 'Avarua', 20000, 11900, Decimal('0.5950')),
 ('Qatar', 'Doha', 599000, 355000, Decimal('0.5927')),
 ('Western Sahara', 'El-Aaiún', 293000, 169000, Decimal('0.5768')),
 ('Bahamas', 'Nassau', 307000, 172000, Decimal('0.5603')),
 ('Seychelles', 'Victoria', 77000, 41000, Decimal('0.5325')),
 ('Cayman Islands', 'George Town', 38000, 19600, Decimal('0.5158')),
 ('Holy See (Vatican City State)',
  'Città del Vaticano',
  1000,
  455,
  Decimal('0.4550')),
 ('Svalbard and Jan Mayen', 'L

Genere una lista de los países que hablan inglés como idioma oficial. Para cada país cuyo idioma oficial es el inglés, liste la fracción de la población que habla inglés y ordene de acuerdo de menor a mayor. 